In [40]:
import time

import numpy as np
import tensorflow as tf

# Conditional and Case Statements

## `tf.cond`

`tf.cond` is basically an `if/else` statement. You provide a boolean predicate and two functions which return tensors. One will run if the predicate is `True`, the other if it is `False`. Here's a simple example:

In [13]:
tf.reset_default_graph()
pred = tf.placeholder(tf.bool)
def run_if_true():
    return tf.add(3, 3)
def run_if_false():
    return tf.square(3)
out = tf.cond(pred, run_if_true, run_if_false)

In [24]:
with tf.Session() as sess:
    choice = np.random.choice([True, False])
    feed_dict = {pred: choice}
    res = sess.run(out, feed_dict)
    print('Choice: {}\tResult: {}'.format(choice, res))

Choice: False	Result: 9


For simple functions, we can use lambdas instead:

In [18]:
tf.reset_default_graph()
pred = tf.placeholder(tf.bool)
out = tf.cond(pred, lambda: tf.add(3, 3), lambda: tf.square(3))

## Stochastic Depth

https://arxiv.org/pdf/1603.09382.pdf

In [108]:
def stochastic_depth_conv2d(inputs, filters, kernel_size, keep_prob, padding='same', activation=tf.nn.relu, name=None):
    with tf.variable_scope(name, 'stochastic_depth_conv'):
        def full_layer():
            return tf.layers.conv2d(inputs, filters, kernel_size, padding=padding, activation=activation)
        def skip_layer():
            if inputs.get_shape().as_list()[-1] != filters:
                return tf.layers.conv2d(inputs, filters, [1, 1], padding=padding, activation=activation)
            else:
                return inputs
        pred = tf.random_uniform([]) < keep_prob
        return tf.cond(pred, full_layer, skip_layer)

In [109]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 228, 228, 3], name='inputs')
keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
conv = stochastic_depth_conv2d(inputs, 32, [3, 3], keep_prob)
conv = stochastic_depth_conv2d(conv, 32, [3, 3], keep_prob)
conv = stochastic_depth_conv2d(conv, 32, [3, 3], keep_prob)
conv = stochastic_depth_conv2d(conv, 64, [3, 3], keep_prob)
conv = stochastic_depth_conv2d(conv, 64, [3, 3], keep_prob)
conv = stochastic_depth_conv2d(conv, 64, [3, 3], keep_prob)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    feed_dict = {
        inputs: np.random.normal(size=[32, 228, 228, 3]),
        keep_prob: 0.5
    }
    start_t = time.time()
    sess.run(conv, feed_dict)
    end_t = time.time()
    print(end_t - start_t)

0.7934935092926025


In [86]:
tf.summary.FileWriter('stochastic_graph', graph=tf.get_default_graph()).close()

# Checking out gradients

In [118]:
def stochastic_depth_conv2d(inputs, filters, kernel_size, keep_prob, padding='same', activation=tf.nn.relu, name=None):
    with tf.variable_scope(name, 'stochastic_depth_conv'):
        def full_layer():
            return tf.layers.conv2d(inputs, filters, kernel_size, padding=padding, activation=activation, name='conv')
        def skip_layer():
            if inputs.get_shape().as_list()[-1] != filters:
                return tf.layers.conv2d(inputs, filters, [1, 1], padding=padding, activation=activation, name='skip')
            else:
                return inputs
        pred = tf.random_uniform([]) < keep_prob
        return tf.cond(pred, full_layer, skip_layer), pred

In [149]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 3, 3, 1], name='inputs')
keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
conv1, pred1 = stochastic_depth_conv2d(inputs, 3, [3, 3], keep_prob, name='conv1')
conv2, pred2 = stochastic_depth_conv2d(conv1, 3, [3, 3], keep_prob, name='conv2')
conv3, pred3 = stochastic_depth_conv2d(conv2, 3, [3, 3], keep_prob, name='conv3')
init = tf.global_variables_initializer()

opt = tf.train.GradientDescentOptimizer(0.05)
var_list = tf.trainable_variables()
grads = opt.compute_gradients(conv3, var_list)

In [150]:
with tf.Session() as sess:
    sess.run(init)
    feed_dict = {
        inputs: np.random.normal(size=[1, 3, 3, 1]),
        keep_prob: 0.5
    }
    g, p1, p2, p3 = sess.run([grads, pred1, pred2, pred3], feed_dict)
    print(p1, p2, p3)
    for var, grad_value in zip(var_list, g):
        grad, value = grad_value
        print('',var.op.name, grad.squeeze(), sep='\n')

False True True

conv1/conv/kernel
[[[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]

conv1/conv/bias
[ 0.  0.  0.]

conv1/skip/kernel
[-0.14867817 -0.25867996  0.49975032]

conv1/skip/bias
[-0.04753564  0.21334063  0.26120389]

conv2/conv/kernel
[[[[-0.23143603  0.45451713  0.36707506]
   [-0.06008153  0.1246832   0.02880499]
   [-0.50861377  0.99886632  0.80669987]]

  [[-0.46919554  0.          0.        ]
   [-0.14349511  0.41343397  0.04362874]
   [-1.03112423  0.          0.        ]]

  [[-0.13635001  0.          0.        ]
   [-0.12059332  0.23890024  0.        ]
   [-0.29964861  0.          0.        ]]]


 [[[-0.12739229  0.50220871  0.20946743]
   [-0.08504266  0.          0.        ]
   [-0.27996275  1.10367548  0.46033463]]

  [[-0.4885253   0.92358738  0.        ]
   [-0.12520075  0.24877444  0.02124467]
   [-1.07360423  2.0297153   0.        ]]

  [[-0.42817545  1.1857729

## Efficiency Comparison

Unfortunately, due to the requirements of TensorFlow's dataflow model, chaining multiple `tf.cond()` statments doesn't provide a performance benefit

In [104]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 228, 228, 3], name='inputs')
keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
conv = stochastic_depth_conv2d(inputs, 32, [3, 3], keep_prob)
for i in range(100):
    conv = stochastic_depth_conv2d(conv, 32, [3, 3], keep_prob)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    feed_dict = {
        inputs: np.random.normal(size=[32, 228, 228, 3]),
        keep_prob: 0.5
    }
    start_t = time.time()
    sess.run(conv, feed_dict)
    end_t = time.time()
    print(end_t - start_t)

0.12723159790039062


In [102]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 228, 228, 3], name='inputs')
conv = tf.layers.conv2d(inputs, 32, [3, 3], padding='same', activation=tf.nn.relu)
for i in range(100):
    conv = tf.layers.conv2d(inputs, 32, [3, 3], padding='same', activation=tf.nn.relu)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    feed_dict = {inputs: np.random.normal(size=[32, 228, 228, 3])}
    start_t = time.time()
    sess.run(conv, feed_dict)
    end_t = time.time()
    print(end_t - start_t)

0.14161086082458496
